## This notebook explores colors using the different flux measurements

#### Standard imports

In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl

#### Change this to get the right location

In [ ]:
# pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'
# if that fails you can use this
# pz_dir = '/global/u2/e/echarles/dx'
pz_dir = '/Users/echarles/pz'

In [ ]:
t = tables_io.read(f'{pz_dir}/data/dp1_v29.0.0/5063/object.parquet')

#### Some useful functions

In [ ]:
def flux_to_mag(fluxes, zero_points=31.4):
    return -2.5 * np.log10(fluxes.clip(0.001, np.inf)) + zero_points

def get_mags(table, bands, band_template):
    mag_dict = {}
    for band_ in bands:
        col = band_template.format(band=band_)
        mags = flux_to_mag(table[col].to_numpy())
        mag_dict[band_] = mags
    return mag_dict

def make_colors(table, bands, band_template):
    colors = {}
    for i, band_ in enumerate(bands[0:-1]):
        col_a = band_template.format(band=band_)
        col_b = band_template.format(band=bands[i+1])
        mag_a = flux_to_mag(table[col_a].to_numpy())
        mag_b = flux_to_mag(table[col_b].to_numpy())
        colors[f"{band_}-{bands[i+1]}"] = mag_a - mag_b
    return colors

In [ ]:
def plot_scatter(t1, t2, bands, mask=None, norm='log'):
    fig = plt.figure()
    axes = fig.subplots(2, 2)
    for i, band in enumerate(bands):
        icol = i%2
        irow = int(i/2)
        axs = axes[irow][icol]
        if mask is None:
            axs.hist2d(t1[band], t2[band], bins=(np.linspace(16, 30, 101), np.linspace(16, 30, 101)), cmap='gray', norm='log')
        else:
            axs.hist2d(t1[band][mask], t2[band][mask], bins=(np.linspace(16, 30, 101), np.linspace(16, 30, 101)), cmap='gray', norm='log')
        axs.plot([16,30],[16,30])
        axs.set_xlabel(f'{band} [mag]')
        axs.set_ylabel(f'{band} [mag]')
    fig.tight_layout()

def plot_resid(t1, t2, bands, mask=None, norm='log'):
    fig = plt.figure()
    axes = fig.subplots(2, 2)
    for i, band in enumerate(bands):
        icol = i%2
        irow = int(i/2)
        axs = axes[irow][icol]
        if mask is None:
            axs.hist2d(t2[band], t1[band] - t2[band], bins=(np.linspace(16, 30, 101), np.linspace(-2, 2, 101)), cmap='gray', norm='log')
        else:
            axs.hist2d(t2[band][mask], t1[band][mask] - t2[band][mask], bins=(np.linspace(16, 30, 101), np.linspace(-2, 2, 101)), cmap='gray', norm='log')
        axs.set_xlabel(f'{band} [mag]')
        axs.set_ylabel(f'Delta {band} [mag]')
    fig.tight_layout()

In [ ]:
def plot_color_scatter(t1, t2, bands, mask=None, norm='log'):
    fig = plt.figure()
    axes = fig.subplots(2, 2)
    for i, band in enumerate(bands):
        icol = i%2
        irow = int(i/2)
        axs = axes[irow][icol]
        if mask is None:
            axs.hist2d(t1[band], t2[band], bins=(np.linspace(-0.5, 2.0, 101), np.linspace(-0.5, 2.0, 101)), cmap='gray', norm=norm)
        else:
            axs.hist2d(t1[band][mask], t2[band][mask], bins=(np.linspace(-0.5, 2.0, 101), np.linspace(-0.5, 2.0, 101)), cmap='gray', norm=norm)
        axs.set_xlabel(f'{band} [mag]')
        axs.set_ylabel(f'{band} [mag]')
    fig.tight_layout()

def plot_color_resid(t1, t2, bands, mask=None, norm='log'):
    fig = plt.figure()
    axes = fig.subplots(2, 2)
    for i, band in enumerate(bands):
        icol = i%2
        irow = int(i/2)
        axs = axes[irow][icol]
        if mask is None:                
            axs.hist2d(t2[band], t1[band] - t2[band], bins=(np.linspace(-0.5, 2.0, 101), np.linspace(-1.0, 1.0, 101)), cmap='gray', norm=norm)
        else:
            axs.hist2d(t2[band][mask], (t1[band] - t2[band])[mask], bins=(np.linspace(-0.5, 2.0, 101), np.linspace(-1.0, 1.0, 101)), cmap='gray', norm=norm)
        axs.set_xlabel(f'{band} [mag]')
        axs.set_ylabel(f'Delta {band} [mag]')
    fig.tight_layout()

#### Define data quality masks

In [ ]:
blendedness = t['i_blendedness'].to_numpy()
blendedness = np.where(np.isnan(blendedness), 0., blendedness).clip(0., 1.)
mask  = np.bitwise_and(
    np.bitwise_and(
        t['i_extendedness'].to_numpy() == 1,
        t['g_psfFlux'].to_numpy() / t['g_psfFluxErr'].to_numpy() > 10,
    ),
    np.bitwise_and(
        t['i_psfFlux'].to_numpy() / t['i_psfFluxErr'].to_numpy() > 10,
        t['r_psfFlux'].to_numpy() / t['r_psfFluxErr'].to_numpy() > 10,
        t['z_psfFlux'].to_numpy() / t['z_psfFluxErr'].to_numpy() > 10,
    ),
)


#### Get the magnitudes

In [ ]:
mags_psf = get_mags(t, 'griz', '{band}_psfFlux')
mags_cModel = get_mags(t, 'griz', '{band}_cModelFlux')
mags_sersic = get_mags(t, 'griz', '{band}_sersicFlux')
mags_1p0 = get_mags(t, 'griz', '{band}_gaap1p0Flux')
mags_3p0 = get_mags(t, 'griz', '{band}_gaap3p0Flux')

#### Make some plots

In [ ]:
#plot_resid(mags_cModel, mags_psf, 'griz')

In [ ]:
#plot_scatter(mags_cModel, mags_psf, 'griz')

In [ ]:
#plot_resid(mags_sersic, mags_psf, 'griz')

In [ ]:
#plot_scatter(mags_sersic, mags_psf, 'griz')

In [ ]:
plot_scatter(mags_1p0, mags_psf, 'griz', mask=mask)

In [ ]:
plot_resid(mags_1p0, mags_psf, 'griz', mask=mask)

In [ ]:
plot_resid(mags_3p0, mags_psf, 'griz', mask=mask)

In [ ]:
plot_scatter(mags_3p0, mags_psf, 'griz', mask=mask)

In [ ]:
plot_scatter(mags_sersic, mags_psf, 'griz', mask=mask)

In [ ]:
plot_resid(mags_sersic, mags_psf, 'griz', mask=mask)

In [ ]:
plot_scatter(mags_cModel, mags_psf, 'griz', mask=mask)

In [ ]:
plot_resid(mags_cModel, mags_psf, 'griz', mask=mask)

In [ ]:
colors_psf = make_colors(t, 'griz', '{band}_psfFlux')
colors_cModel = make_colors(t, 'griz', '{band}_cModelFlux')
colors_1p0 = make_colors(t, 'griz', '{band}_gaap1p0Flux')
colors_3p0 = make_colors(t, 'griz', '{band}_gaap3p0Flux')
colors_sersic = make_colors(t, 'griz', '{band}_sersicFlux')

In [ ]:
plot_color_resid(colors_cModel, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask)

In [ ]:
plot_color_scatter(colors_cModel, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask)

In [ ]:
plot_color_scatter(colors_3p0, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask, norm="log")

In [ ]:
plot_color_resid(colors_cModel, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask)

In [ ]:
plot_color_resid(colors_1p0, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask)

In [ ]:
plot_color_resid(colors_sersic, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask)

In [ ]:
plot_color_resid(colors_3p0, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask, norm="log")